In [1]:
import numpy as np

sample_num = 64
sample_feature = 1000
hidden_units = 100
# 每个unit的输出维度？
hidden_output_dim = 10

learning_rate = 1e-6

x = np.random.randn(sample_num, sample_feature)
y = np.random.randn(sample_num, hidden_output_dim)

# 神经网络的权重
# 每个hidden unit有一个vector对应feature, 1000个feature, 1000个权重，每个unit 1000个权重
w1 = np.random.randn(sample_feature, hidden_units)
# 与上面类似。
w2 = np.random.randn(hidden_units, hidden_output_dim)

print("x.shape\n", x.shape)
print("y.shape\n", y.shape)
print("w1.shape\n", w1.shape)
print("w2.shape\n", w2.shape)

x.shape
 (64, 1000)
y.shape
 (64, 10)
w1.shape
 (1000, 100)
w2.shape
 (100, 10)


In [ ]:
"""
前向传播 
"""
def forward(x): 
    # 1d数组内积，2d数学矩阵乘法，注意不一样。
    # hidden unit的结果 
    h = x.dot(w1)
    # h.shape (64,100)
    # element-wise maximum，可以直接作用到2d情况
    relu_output = np.maximum(h, 0)
    # relu_output.shape (64, 100)
    y_predict = relu_output.dot(w2)
    # yhat.shape (64, 100)
    return y_predict
    
"""
loss function - MSE
BGD - 在更新参数时使用所有的样本来进行更新。
"""
def loss(y_predict, y):
    y_diff = y_predict - y
    # shape = (64, 100)
    y_diff_square = np.square(y_diff)
    # shape - ?
    return y_diff_square.sum()

"""
反向传播函数，更新w1, w2
实际上这里的loss函数是
loss = (y - w2 * relu(w1 * x))^2，要想这个尽量小，loss对w2, w1求导。
loss对w2求导：
2(y - w2 * relu(w1 * x)) * relu(w1 * x), 而w2 * relu(w1 * x)其实就是y_predict，前面已经结算完了。
"""
def back_propagation(y_predict, y, x, w1, w2):
    grad_y_predict = 2.0 * (y_predict - y) 
    grad_w2 = np.maximum(x.dot(w1), 0).T.dot(grad_y_predict)
    w2 -= learning_rate * grad_w2 
    
    
    